In [ ]:
!pip install selenium

     |████████████████████████████████| 954 kB 9.0 MB/s 
     |████████████████████████████████| 356 kB 26.6 MB/s 
     |████████████████████████████████| 138 kB 43.3 MB/s 
     |████████████████████████████████| 3.5 MB 20.4 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

In [34]:
# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=hide+on+bush'

# selenium이 제어할 chrome을 실행합니다.
# driver = webdriver.Chrome("/Users/namshin/chromedriver")

In [ ]:
# step1
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

# step2
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#step3
driver = webdriver.Chrome('chromedriver',options=options)

In [32]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(faker_opgg_url)

In [33]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.


In [35]:
# 게임이 모두 로딩된 페이지의 html 코드를 faker_total_html로 저장합니다.
faker_total_html = driver.page_source

# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [36]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
faker_total_soup = BeautifulSoup(faker_total_html, 'html.parser')

In [37]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_champions = []
faker_kills = []
faker_deaths = []
faker_assists = []
faker_results = []

In [41]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
faker_total_games_html = faker_total_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(faker_total_games_html)
print(total_game_len)

609


In [39]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(total_game_len):
    faker_champions.append(list(faker_total_games_html[i].select_one('div.ChampionName').
                                               stripped_strings)[0])
    faker_kills.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Kill').
                                               stripped_strings)[0])
    faker_deaths.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    faker_assists.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Assist').
                                               stripped_strings)[0])
    faker_results.append(list(faker_total_games_html[i].select_one('div.GameStats div.GameResult').
                                               stripped_strings)[0])

In [40]:
# DataFrame으로 변환 후 출력
faker_total_df = pd.DataFrame([faker_champions,
                                faker_results,
                                faker_kills,
                                faker_deaths,
                                faker_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_total_df

,champion,result,kills,deaths,assists
0,Ekko,Victory,4,7,11
1,LeBlanc,Defeat,2,2,5
2,LeBlanc,Victory,6,4,4
3,Taliyah,Defeat,5,10,7
4,LeBlanc,Victory,8,5,10
...,...,...,...,...,...
604,Akali,Defeat,5,3,3
605,Akali,Defeat,6,7,3
606,Viktor,Victory,6,1,7
607,Akali,Victory,2,2,2
